<a href="https://colab.research.google.com/github/bm627/Masters-Sweepstake/blob/main/Masters_Sweepstake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import time 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from datetime import datetime

from google.colab import auth
auth.authenticate_user()

from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# Use the name of your Google Sheet here instead of 'Elastic_Beams_Template'. 
spreadsheet = gc.open('Masters Sweepstake Live')

# Load the worksheet.
worksheet = spreadsheet.worksheet('Entry')

# Get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame.
df = pd.DataFrame.from_records(rows)

CUGCandPlayer = df.to_dict()

def updateScore():
  url = "https://www.golfchannel.com/api/v2/events/19956/leaderboard"

  payload = ""
  response = requests.request("GET", url, data=payload)

  jsondata = json.loads(response.text)

  playersAndScores = {}

  for i in range(len(jsondata['result']['golfers'])):
      firstName = jsondata['result']['golfers'][i]['firstName']
      lastName = jsondata['result']['golfers'][i]['lastName']
      score = jsondata['result']['golfers'][i]['overallPar']
      name = firstName + ' ' + lastName
      newDict = {name:score}
      playersAndScores.update(newDict)
  return(playersAndScores)

def create7List(CUGCandPlayer):
  listOf7List = []
  for i in range(len(CUGCandPlayer)):
    playerAndChoices = CUGCandPlayer.get(i)
    CUGCandChoices = []
    for i in range(len(playerAndChoices)):
      CUGCandChoices.append(playerAndChoices.get(i))
    listOf7List.append(CUGCandChoices)
  return(listOf7List)

def createPersonAndChoiceDict():
    sevenList = create7List(CUGCandPlayer)
    personAndChoiceDict = {}
    for i in range(len(sevenList)):
      person = sevenList[i][0]
      choices = sevenList[i][1:7]
      newDict = {person:choices}
      personAndChoiceDict.update(newDict)
    return(personAndChoiceDict)

def CUGCScores(personAndChoiceDict, playersAndScores):
  scoreDict = {}
  personAndChoiceDict = createPersonAndChoiceDict()
  for person in personAndChoiceDict:
    chosenPlayers = personAndChoiceDict.get(person)
    scoreList = []
    for player in chosenPlayers:
      if player in playersAndScores:
        scoreList.append(playersAndScores.get(player))
      else:
        pass

    #Converting list of string scores to integer scores, sorting, taking top 4 and adding to give total
    scoreInts = []
    for score in scoreList:
      scoreInts.append(int(score))

    scoreInts.sort()

    topFour = scoreInts[0:4]

    total = sum(topFour)

    #Converting player and score to dictionary form
    newDict = {person:total}
    scoreDict.update(newDict)
  #Sorting dictionary to find leaderboard
  sortedScoreDict = dict(sorted(scoreDict.items(), key=lambda x:x[1]))
  return sortedScoreDict

def uploadScores(data):
  #Creating the list of people
  people = []
  for person in data.keys():
      people.append(person)
  
  #Creating the corresponding list of scores
  scores = []
  for score in data.values():
      scores.append(score)

  # Create empty dataframe
  df = pd.DataFrame()

  # Create a column
  df['CUGC'] = people
  df['Scores'] = scores
  
  # Use the name of your Google Sheet here instead of 'Elastic_Beams_Template'. 
  spreadsheet = gc.open('Masters Sweepstake Live')

  # Load the worksheet.
  worksheet = spreadsheet.worksheet('Leaderboard')
  set_with_dataframe(worksheet, df) 

def timeUpdated():
  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")
  print('Updated at: '+current_time)
  
if __name__ == "__main__":
  while True:
    personAndChoiceDict = createPersonAndChoiceDict()
    playersAndScores = updateScore()
    data = CUGCScores(personAndChoiceDict, playersAndScores)
    uploadScores(data)
    timeUpdated()
    time.sleep(300)

Updated at: 10:32:25
